<a href="https://colab.research.google.com/github/JhonnyLimachi/IA_en_Salud/blob/main/Aula_016_Limpeza_de_dados%2C_outliers_e_valores_ausentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center"><img src="https://raw.githubusercontent.com/carlosfab/escola-data-science/master/img/eds.png" height="100px"></p>

# Limpeza de dados, outliers e valores ausentes

Neste projeto, vamos focar em uma habilidade básica, mas essencial de um Cientista de Dados. Saber lidar com valores ausentes, lidar com *outliers*, transformar os dados. Essas são algumas das atividades que fazem parte do checklist de todos os projetos que executamos.


Esses processos podem ter um impacto enorme nos resultados, desde a parte da análise dos dados até os modelos de *Machine Learning*.
<p align="center"><img src="https://image.freepik.com/free-vector/group-analysts-working-graphs_1262-21249.jpg
"></p>


Aqui, veremos o que buscar entender nos dados, os passos a dar, como identificar, e tratar *outliers*, as melhores práticas e métodos para lidar com dados ausentes, e por fim, converter os dados para os formatos necessários.


## O Que Analisar?

Primeiro de tudo, precisamos entender o que são bons dados, para que saibamos os passos que precisamos tomar para ter o melhor conjunto de dados possível, a partir dos dados que estamos utilizando.

* Tipo de Dados
  * Coluna de data em formato `datetime`, coluna de valores monetários em `float`.
* *Range* dos Dados
  * Meses de 1 a 12, dias do mês de 1 a 31.
* Dados Obrigatórios
  * Algumas colunas não podem estar vazias
* Dados Únicos
  * CPF, RG, CNPJ, ID de usuário.
* Dados Categóricos
  * Gênero Masculino ou Feminino.
* Padrões regulares
  * (61) 9 8765-4321
* Validade Entre os Campos
  * Data de saída não ser anterior à data de entrada.


In [ ]:
# importando os pacotes necessários
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# configurando a visualização
sns.set_style()
%matplotlib inline

## Os Dados

Utilizaremos mais de um conjunto de dados nesse projeto, mas o primeiro deles será do mesmo grupo usado no Projeto do Módulo 2 do curso Data Science na Prática.

Nesse projeto, analisamos dados sobre a violência no Rio de Janeiro.

<p align="center"><img src="https://image.freepik.com/free-vector/brazilian-carnival-concept-with-dancing-people-nature_1284-27444.jpg
", width="50%"></p>


Esse conjunto de dados é interessante pois está organizado em números de crimes por mês, desde janeiro de 1991, mas alguns dos crimes só começaram a ser registrados algum tempo depois, como veremos abaixo.

In [ ]:
# importando os dados
df = pd.read_csv('https://raw.githubusercontent.com/carlosfab/curso_data_science_na_pratica/master/modulo_02/violencia_rio.csv', sep=',')

# verificando as dimensões
print('Dimensões do Dataset',
    '\nVariáveis: ',df.shape[1], "\n"
      'Entradas: ', df.shape[0])

# verificando as primeiras entradas
df.head()


Dimensões do Dataset 
Variáveis:  56 
Entradas:  344


,vano,mes,hom_doloso,lesao_corp_morte,latrocinio,hom_por_interv_policial,tentat_hom,lesao_corp_dolosa,estupro,hom_culposo,...,pessoas_desaparecidas,encontro_cadaver,encontro_ossada,pol_militares_mortos_serv,pol_civis_mortos_serv,indicador_letalidade,indicador_roubo_rua,indicador_roubo_veic,registro_ocorrencias,fase
0,1991,1,657,NaN,15,NaN,162,3051,NaN,NaN,...,NaN,217,NaN,NaN,NaN,672,1348,1174,NaN,3
1,1991,2,732,NaN,17,NaN,175,3421,NaN,NaN,...,NaN,209,NaN,NaN,NaN,749,1395,1097,NaN,3
2,1991,3,713,NaN,25,NaN,216,3613,NaN,NaN,...,NaN,188,NaN,NaN,NaN,738,1385,1265,NaN,3
3,1991,4,634,NaN,20,NaN,200,3211,NaN,NaN,...,NaN,140,NaN,NaN,NaN,654,1540,1415,NaN,3
4,1991,5,650,NaN,20,NaN,146,3051,NaN,NaN,...,NaN,78,NaN,NaN,NaN,670,1266,1449,NaN,3


In [ ]:
# verificando o final do dataset
df.tail()

,vano,mes,hom_doloso,lesao_corp_morte,latrocinio,hom_por_interv_policial,tentat_hom,lesao_corp_dolosa,estupro,hom_culposo,...,pessoas_desaparecidas,encontro_cadaver,encontro_ossada,pol_militares_mortos_serv,pol_civis_mortos_serv,indicador_letalidade,indicador_roubo_rua,indicador_roubo_veic,registro_ocorrencias,fase
339,2019,4,360,1.0,11,124.0,466,5573,483.0,172.0,...,408.0,22,3.0,1.0,0.0,496,11040,3755,67797.0,3
340,2019,5,345,2.0,15,172.0,478,4958,465.0,145.0,...,390.0,20,1.0,0.0,0.0,534,11384,3649,68336.0,3
341,2019,6,332,3.0,8,153.0,436,4769,414.0,152.0,...,403.0,20,1.0,3.0,0.0,496,9551,3115,61202.0,3
342,2019,7,309,5.0,10,194.0,399,4740,402.0,140.0,...,400.0,32,7.0,0.0,0.0,518,10071,3198,65817.0,2
343,2019,8,318,1.0,6,170.0,457,4760,460.0,156.0,...,367.0,27,8.0,2.0,0.0,495,9912,3181,65285.0,2


In [ ]:
# modificando o tipo
df.lesao_corp_morte[343].astype('int64')

1

In [ ]:
# pandas usa floats quando tem dados ausentes
df.dtypes

,0
vano,int64
mes,int64
hom_doloso,int64
lesao_corp_morte,float64
latrocinio,int64
hom_por_interv_policial,float64
tentat_hom,int64
lesao_corp_dolosa,int64
estupro,float64
hom_culposo,float64


## Como lidar com Dados Ausentes

A resposta é: Depende!

Que tipo de dado está ausente? Em qual proporção? De forma aleatória? Todos esses são aspectos que precisamos levar em consideração ao tratar dados ausentes.

Tomando os dados do RJ como exemplo, vemos que temos algumas colunas com quase todos os dados ausentes, mas qual o motivo disso? De onde esses dados são extraídos e qual o processo de coleta deles? Dados ausentes implicam algum significado?

Nesse caso, nossa teoria mais predominante é que os dados não começaram a ser registrados até uma determinada data, e após isso, a coleta foi feita de forma efetiva.

In [ ]:
# mostrando a quantidade de dados ausentes por variável
(df.isnull().sum()).sort_values(ascending=False)

,0
furto_bicicleta,276
roubo_bicicleta,276
cmba,180
posse_drogas,180
trafico_drogas,180
apreensao_drogas_sem_autor,180
apf,180
aaapai,180
cmp,180
furto_celular,144


In [ ]:
# dados ausentes por ano
df.set_index('ano').isna().sum(level=0)

KeyError: "None of ['ano'] are in the columns"

In [ ]:
# visualizando
df[273:300]

## Tratando os Dados

No caso desse conjunto de dados específico, o ideal é analisar os dados apenas do período em que se tem dados. Especialmente em algumas variáveis onde o volume é muito grande, qualquer tipo de preenchimento poderia enviesar os dados de forma que a análise deixasse de ser relevante.

Para outros casos, podemos considerar as seguintes hipóteses:

* Excluir
  * Se os dados ausentes estão em pequeno número,ocorrem aleatoriamente, e a ausência não carrega significado, é melhor excluir a linha. No caso da coluna, se ainda for possível analisar alguma parte dela, use-a, como é o caso aqui. Mas para algumas situações, o ideal é excluir a coluna.

* Preencher
  * Preencher as entradas com dados ausentes com valores estatísticos como a média, mediana, moda ou zeros.
  * A média é mais útil quando a distribuição dos dados é normal. Em dados com distribuição mais enviesada (*skewed*), a mediana é uma solução mais robusta, pois ela é menos sensível a outliers.
  * Uma `Regressão Linear` também pode ser útil, apesar de sensível a outliers, podem nos ajudar a inserir valores que nos ajudem.
  * Indetificar a entrada ausente com algum valor que indique isso pode ser mais informativo, quando a ausência representa valor. Por exemplo, em dados numéricos preencher com zero, e em categóricos criar uma categoria "Desconhecido". Atenção, pois os zeros não podem ser levados em consideração em análises estatísticas.

## Tratando Outliers

<p align="center"><img src="https://miro.medium.com/max/18000/1*2c21SkzJMf3frPXPAR_gZA.png
", width="50%"></p>


*Outliers* são pontos discrepantes, que estão destoando do padrão do conjunto de dados.

É muito importante conseguir identificar e tratar esses outliers, pois eles podem nos mostrar uma imagem incorreta dos nossos dados.

Podemos identificar um outlier de diversas formas, entre elas podemos citar:

* IQR Score
* Boxplots
* Scatter plots
* Z-Score

Vamos ver na prática o processo completo de limpeza de dados, tratando dados ausentes, convertendo os dados para o formato correto, e tratando dos outliers.

## Airbnb - NYC

Para esse estudo, faremos o tratamento dos dados do Airbnb referentes à cidade de Nova Iorque.

<center><img alt="New York City" width="50%" src="https://image.freepik.com/free-vector/future-metropolis-downtown-modern-city-business-center-cartoon-background_33099-1466.jpg"></center>

In [ ]:
# importando os dados
df_nyc = pd.read_csv('https://raw.githubusercontent.com/rafaelnduarte/eds_outliers/master/nyc.csv', index_col=0)

# verificando as dimensões
print('Dimensões do Dataset',
    '\nVariáveis: ',df_nyc.shape[1], "\n"
      'Entradas: ', df_nyc.shape[0])

# verificando as primeiras entradas
df_nyc.head()

In [ ]:
# verificando os tipos
df_nyc.dtypes

In [ ]:
# transformando as colunas
df_nyc[['price','latitude', 'longitude']] = df_nyc[['price','latitude', 'longitude']].astype('object')
# verificando o resultado;l
df_nyc.dtypes

In [ ]:
# verificando dados ausentes
(df_nyc.isnull().sum()).sort_values(ascending=False)

Seguindo o que falamos anteriormente, colunas com baixo poder preditivo e grande quantidade de dados faltantes podem ser excluídas.

Em relação às outras entradas, estão em poucas entradas, não parecem ter algum tipo de relação entre elas, e não parecem ter grande poder preditivo. Mais uma vez, vamos fazer a exclusão. Porém, aqui vamos excluir as entradas.

In [ ]:
# excluindo colunas com dados faltantes
df_nyc.drop(columns=['reviews_per_month', 'last_review'], inplace=True)

# excluindo entradas com dados faltantes
df_nyc.dropna(axis=0, inplace=True)

# convertendo os tipos de dados
df_nyc[['price','latitude', 'longitude']] = df_nyc[['price','latitude', 'longitude']].astype('float')

Feita a limpeza, é hora de conferir os resultados.

In [ ]:
# verificando o resultado
(df_nyc.isnull().sum()).sort_values(ascending=False)

## Limpando Outliers

Para tratar dos outliers desse conjunto de dados, iremos analisar a distribuição estatística, plotar boxplots e calcular os limites utilizando a regra do IQR Score.

Primeiramente, vamos lembrar o que é o IQR.

O IQR é calculado subtraindo o Terceiro Quartil (75%) pelo Primeiro Quartil (25%).

# IQR = Q3 - Q1

Vamos dar uma olhada nos nossos dados e ver o que identificamos.

In [ ]:
# verificando a distribuição estatística
df_nyc.describe().round(1)

Aqui, algumas coisas já chamam a nossa atenção, como por exemplo:

* A variável `price` tem o mínimo em 0.
* Lembrando que a variável `price` trata do preço da diária dos imóveis em moeda local (USD), estamos vendo que o Q3 está em 175 dólares, mas o máximo está em 10 mil dórales. Claramente, há outliers por aqui.
* A variável `minimum_nights` tem como seu máximo o valor 1250, sendo que o Q3 está em 6. Claramente temos outliers nessa variável.
* As variáveis `number_of_reviews`, `calculated_host_listings_count` e `availability_365` também podem conter outliers, mas não vamos nos preocupar com elas agora.

In [ ]:
# verificando as distribuições
df_nyc.hist(figsize=(20,15), grid=False);

Verificando os histogramas, conseguimos ver claramente que temos outliers presentes. Para tratá-los vamos seguir os seguintes passos:

* Definir o Q1 e Q3 para as variáveis que serão limpas.
* Calcular o IQR para as variáveis.
* Definir o limite superior e inferior para cortar os outliers.
* Remover os outliers.


In [ ]:
# identificando os outliers para a variável price
q1_price = df_nyc.price.quantile(.25)
q3_price = df_nyc.price.quantile(.75)
IQR_price = q3_price - q1_price
print('IQR da variável price: ', IQR_price)

# definindo os limites
sup_price = q3_price + 1.5 * IQR_price
inf_price = q1_price - 1.5 * IQR_price

print('Limite superior de price: ', sup_price)
print('Limite inferior de price: ', inf_price)


Aqui podemos ver que, apesar de não termos outliers na parte inferior, continuamos tendo valores iguais a zero, que precisam ser tratados.

Vamos plotar um boxplot para visualizarmos a diferença feita pela limpeza.

In [ ]:
# verificando o conjunto original
fig, ax = plt.subplots(figsize=(15,3))
df_nyc.price.plot(kind='box', vert=False);
ax.set_title('Dataset Original - price')
plt.show()
print("O dataset possui {} colunas".format(df_nyc.shape[0]))
print("{} Entradas acima de 335.5".format(len(df_nyc[df_nyc.price > 335.5])))
print("Representam {:.2f}% do dataset".format((len(df_nyc[df_nyc.price > 335.5]) / df_nyc.shape[0])*100))

In [ ]:
# identificando os outliers para a variável minimum_nights
q1_minimum_nights = df_nyc.minimum_nights.quantile(.25)
q3_minimum_nights = df_nyc.minimum_nights.quantile(.75)
IQR_minimum_nights = q3_minimum_nights - q1_minimum_nights
print('IQR da variável minimum_nights: ', IQR_minimum_nights)

# definindo os limites
sup_minimum_nights = q3_minimum_nights + 1.5 * IQR_minimum_nights
inf_minimum_nights = q1_minimum_nights - 1.5 * IQR_minimum_nights

print('Limite superior de minimum_nights: ', sup_minimum_nights)
print('Limite inferior de minimum_nights: ', inf_minimum_nights)

In [ ]:
# verificando o conjunto original
fig, ax = plt.subplots(figsize=(15,3))
df_nyc.minimum_nights.plot(kind='box', vert=False);
ax.set_title('Dataset Original - minimum_nights')
plt.show()
print("O dataset possui {} colunas".format(df_nyc.shape[0]))
print("{} Entradas acima de 335.5".format(len(df_nyc[df_nyc.minimum_nights > 12.0])))
print("Representam {:.2f}% do dataset".format((len(df_nyc[df_nyc.minimum_nights > 12.0]) / df_nyc.shape[0])*100))

In [ ]:
# limpando o dataset
df_clean = df_nyc.copy()

df_clean.drop(df_clean[df_clean.price > 335.5].index, axis=0, inplace=True)
df_clean.drop(df_clean[df_clean.price == 0.0].index, axis=0, inplace=True)
df_clean.drop(df_clean[df_clean.minimum_nights > 12].index, axis=0, inplace=True)

print('Shape antes da limpeza: ', df_nyc.shape)
print('Shape após a limpeza: ',df_clean.shape)


In [ ]:
# plotando novamente o boxplot original
fig, ax = plt.subplots(figsize=(15,3))
df_nyc.price.plot(kind='box', vert=False);
ax.set_title('Dataset Original - price')
plt.show()
print("O dataset possui {} colunas".format(df_nyc.shape[0]))
print("{} Entradas acima de 335.5".format(len(df_nyc[df_nyc.price > 335.5])))
print("Representam {:.2f}% do dataset".format((len(df_nyc[df_nyc.price > 335.5]) / df_nyc.shape[0])*100))


In [ ]:
# verificando o conjunto limpo
fig, ax = plt.subplots(figsize=(15,3))
df_clean.price.plot(kind='box', vert=False);
ax.set_title('Dataset Limpo - price')
plt.show()
print("Tamanho original: {} colunas".format(df_nyc.shape[0]))
print("Tamanho atual: {} colunas".format(df_clean.shape[0]))

Aqui podemos ver o resultado da limpeza.

O boxplot mostra alguns dados como outliers, entretanto, esses estão sendo calculados em relação ao novo dataset, e nossa limpeza levou em consideração os quartis do dado original.

Para garantirmos que não estamos lidando com outliers que vão prejudicar nossa análise, vamos checar os histogramas novamente.


In [ ]:
# verificando as distribuições
df_clean.hist(figsize=(20,15), grid=False);

In [ ]:
# verificando a distribuição estatística dos dados limpos
df_clean.describe().round(1)

Agora conseguimos ter uma ideia bem melhor da distribuição dos nossos dados.

Alguns destaques:

* A mediana da variável `price` foi pouquíssimo afetada pela limpeza dos outliers, mostrando mais uma vez a robustez desse atributo como solução para dados ausentes.
* Agora, temos dados que respeitam as regras definidas no início do notebook, onde vimos o que são bons dados.
* A média da variável `price` foi reduzida drásticamente, enfatizando a sensibilidade desse atributo em relação aos outliers.

## Informações Importantes

* Modelos lineares são mais sensíveis aos outliers. Ao trabalhar com modelos desse tipo é essencial que o trabalho com outliers seja feito com bastante atenção. (Linear Regression, Logistic Regression)

* Modelos baseados em árvores de decisão são menos sensívels a outliers. (Decision Trees, Random Forest, SVM, KNN)

* Técnicas de pré-processamento como Normalização e Padronização ajudam em relação aos outliers.